In [1]:
import os
print("Current Working Directory:", os.getcwd())  # Prints the directory where Python is looking for the file
print("Files in Directory:", os.listdir())  # Lists all files in the current directory


Current Working Directory: c:\Users\vudut\OneDrive\Desktop\Python\MINI Project
Files in Directory: ['.git', 'CNN.ipynb', 'Data Sets', 'FINAL_1.ipynb', 'logs', 'particle_images', 'PPT & Images', 'preprocessed_data.csv', 'README.md', 'Simulation.ipynb', 'wandb', 'xgboost_model.onnx', 'xgboost_optimized.onnx']


In [ ]:
import pandas as pd

# Load the preprocessed dataset
df_combined = pd.read_csv("preprocessed_data.csv")

# Verify the data
print(df_combined.head())  # Check the first few rows
print(f"Dataset shape: {df_combined.shape}")  # Print dataset size


In [2]:
import tensorflow as tf

# Check available GPUs (should list DirectML GPU)
print("Available Devices:", tf.config.list_physical_devices())

# Force TensorFlow to use DirectML
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices("GPU")[1], True)
print("Using TensorFlow DirectML with GPU")


Available Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Using TensorFlow DirectML with GPU


In [ ]:
import pandas as pd

# Load the preprocessed dataset
df_combined = pd.read_csv("preprocessed_data.csv")

# Verify data
print(df_combined.head())  # Check the first few rows


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from multiprocessing import Pool

# Create a directory to store generated images
os.makedirs("particle_images", exist_ok=True)

# Load the preprocessed dataset
df_combined = pd.read_csv("preprocessed_data.csv")

# Convert Pandas DataFrame to NumPy arrays (DirectML-compatible)
lep_eta = df_combined["lep_eta"].values.astype(np.float32)
lep_phi = df_combined["lep_phi"].values.astype(np.float32)
lep_pt = df_combined["lep_pt"].values.astype(np.float32)

# Function to generate images
def generate_image(args):
    idx, eta, phi, pt = args
    fig, ax = plt.subplots(figsize=(2, 2))
    scatter = ax.scatter(eta, phi, c=pt, cmap="inferno")

    ax.set_xlim(-3, 3)
    ax.set_ylim(-np.pi, np.pi)
    ax.axis("off")

    plt.savefig(f"particle_images/{idx}.png", bbox_inches="tight", pad_inches=0)
    plt.close()

# **Use multiprocessing to speed up image generation**
if __name__ == '__main__':
    with Pool(processes=os.cpu_count()) as pool:
        pool.map(generate_image, zip(range(len(lep_eta)), lep_eta, lep_phi, lep_pt))

print("✅ Image generation completed successfully!")


In [1]:
import os

# Set necessary environment variables
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
# Disable eager execution to avoid early initialization
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

# Only then import tensorflow
import tensorflow as tf

# Make sure TF doesn't eagerly initialize the context
tf.compat.v1.disable_eager_execution()

# Configure memory growth
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    print(f"Found GPU devices: {physical_devices}")
    for device in physical_devices:
        try:
            tf.config.experimental.set_memory_growth(device, True)
            print(f"Memory growth enabled for {device}")
        except Exception as e:
            print(f"Error setting memory growth: {e}")

# Your remaining imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load preprocessed dataset
df_combined = pd.read_csv("preprocessed_data.csv")

# Reduce dataset: Sample 50,000 images
df_sampled = df_combined.sample(n=50000, random_state=42)

# Convert to NumPy arrays - don't use TensorFlow tensors for the matplotlib workflow
lep_eta = df_sampled["lep_eta"].values
lep_phi = df_sampled["lep_phi"].values
lep_pt = df_sampled["lep_pt"].values

# Create directory for images
os.makedirs("particle_images", exist_ok=True)

# Batch size for processing
batch_size = 100

# Process in batches without TensorFlow
for i in range(0, len(lep_eta), batch_size):
    end_idx = min(i + batch_size, len(lep_eta))
    
    for j in range(i, end_idx):
        # Generate and save image using matplotlib directly
        fig, ax = plt.subplots(figsize=(2, 2))
        scatter = ax.scatter(lep_eta[j], lep_phi[j], c=lep_pt[j], cmap="inferno")
        
        ax.set_xlim(-3, 3)
        ax.set_ylim(-np.pi, np.pi)
        ax.axis("off")
        
        plt.savefig(f"particle_images/{j}.png", bbox_inches="tight", pad_inches=0)
        plt.close()
    
    print(f"Processed images {i} to {end_idx-1}")

print("✅ Image generation completed!")

Found GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Memory growth enabled for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Memory growth enabled for PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')
Processed images 0 to 99
Processed images 100 to 199
Processed images 200 to 299
Processed images 300 to 399
Processed images 400 to 499
Processed images 500 to 599
Processed images 600 to 699
Processed images 700 to 799
Processed images 800 to 899
Processed images 900 to 999
Processed images 1000 to 1099
Processed images 1100 to 1199
Processed images 1200 to 1299
Processed images 1300 to 1399
Processed images 1400 to 1499
Processed images 1500 to 1599
Processed images 1600 to 1699
Processed images 1700 to 1799
Processed images 1800 to 1899
Processed images 1900 to 1999
Processed images 2000 to 2099
Processed images 2100 to 2199
Processed images 2200 to 2299

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Ensure images are correctly stored in class subdirectories (e.g., particle_images/class_name/)
image_dir = "particle_images/"
target_dir = "particle_images/processed/"
os.makedirs(target_dir, exist_ok=True)

# Convert any non-PNG images to PNG format
for img_name in tqdm(os.listdir(image_dir)):
    img_path = os.path.join(image_dir, img_name)

    # Ignore non-image files
    if not img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        continue

    # Read image
    img = cv2.imread(img_path)
    if img is None:
        continue  # Skip corrupted images

    # Convert & Save as PNG
    save_path = os.path.join(target_dir, f"{os.path.splitext(img_name)[0]}.png")
    cv2.imwrite(save_path, img)

print("✅ Image verification and conversion completed!")


100%|██████████| 50001/50001 [04:47<00:00, 173.74it/s]


✅ Image verification and conversion completed!


In [4]:
import os
import shutil
from tqdm import tqdm

# Paths
source_dir = "particle_images/"
processed_dir = "particle_images/processed/"
class_0_dir = os.path.join(processed_dir, "class_0")
class_1_dir = os.path.join(processed_dir, "class_1")

# Ensure class directories exist
os.makedirs(class_0_dir, exist_ok=True)
os.makedirs(class_1_dir, exist_ok=True)

# Move images into class folders
for idx, img_name in enumerate(tqdm(os.listdir(source_dir))):
    img_path = os.path.join(source_dir, img_name)

    # Only process valid image files
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        if idx % 2 == 0:
            shutil.move(img_path, os.path.join(class_0_dir, img_name))  # Assign half images to class_0
        else:
            shutil.move(img_path, os.path.join(class_1_dir, img_name))  # Assign half to class_1

print("✅ Image organization completed: Files moved into class-labeled folders.")


100%|██████████| 50001/50001 [00:20<00:00, 2405.00it/s]


✅ Image organization completed: Files moved into class-labeled folders.


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Ensure TensorFlow DirectML uses GPU-0
print("Available GPUs:", tf.config.list_physical_devices('GPU'))
tf.device('/GPU:0')

# Define dataset path
dataset_path = r"C:\Users\vudut\OneDrive\Desktop\Python\MINI Project\particle_images\processed"  # Use the corrected folder

# Load dataset
batch_size = 32
img_size = (64, 64)  # Resize for CNN

train_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

# Prefetch for performance boost
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

print("✅ Dataset Loaded Successfully & Using GPU-0")


Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Found 50000 files belonging to 2 classes.
Using 40000 files for training.
Found 50000 files belonging to 2 classes.
Using 10000 files for validation.
✅ Dataset Loaded Successfully & Using GPU-0


In [6]:
from tensorflow.keras import layers, models

# Force GPU usage
with tf.device('/GPU:0'):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3)),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(2, activation="softmax")  # Assuming binary classification (signal vs. background)
    ])

    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    model.summary()

# Train Model
with tf.device('/GPU:0'):
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=10
    )

print("✅ CNN Training Completed Successfully!")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 128)        0

## Using weights and baises

In [4]:
import wandb
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

# Initialize W&B project
wandb.init(project="particle_cnn", name="initial_training")

# Define hyperparameters
config = wandb.config
config.batch_size = 32
config.epochs = 10
config.learning_rate = 0.001

# Load dataset (replace with your dataset path)
dataset_path = "particle_images/processed"

# Load dataset with GPU acceleration
with tf.device('/GPU:0'):
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        dataset_path,
        validation_split=0.2,
        subset="training",
        seed=42,
        image_size=(64, 64),
        batch_size=config.batch_size
    )

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        dataset_path,
        validation_split=0.2,
        subset="validation",
        seed=42,
        image_size=(64, 64),
        batch_size=config.batch_size
    )

# Define CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train model with W&B logging
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=config.epochs,
    callbacks=[wandb.keras.WandbCallback()]  # W&B auto-logs training metrics
)

# Evaluate the model
test_loss, test_acc = model.evaluate(val_ds)
wandb.log({"Test Accuracy": test_acc, "Test Loss": test_loss})

# Finish the W&B run
wandb.finish()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vuduthasaipraneetham (praneetham) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Found 50000 files belonging to 2 classes.
Using 40000 files for training.
Found 50000 files belonging to 2 classes.
Using 10000 files for validation.


wandb: WARNING WandbCallback is deprecated and will be removed in a future release. Please use the WandbMetricsLogger, WandbModelCheckpoint, and WandbEvalCallback callbacks instead. See https://docs.wandb.ai/guides/integrations/keras for more information.
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/10
1249/1250 [============================>.] - ETA: 0s - loss: 0.8113 - accuracy: 0.5010

INFO:tensorflow:Assets written to: c:\Users\vudut\OneDrive\Desktop\Python\MINI Project\wandb\run-20250228_233740-5zcvrrkr\files\model-best\assets


...

1250/1250 [==============================] - 23s 18ms/step - loss: 0.6932 - accuracy: 0.5008 - val_loss: 0.6932 - val_accuracy: 0.4925
Epoch 3/10
1249/1250 [============================>.] - ETA: 0s - loss: 0.6932 - accuracy: 0.5000

INFO:tensorflow:Assets written to: c:\Users\vudut\OneDrive\Desktop\Python\MINI Project\wandb\run-20250228_233740-5zcvrrkr\files\model-best\assets


INFO:tensorflow:Assets written to: c:\Users\vudut\OneDrive\Desktop\Python\MINI Project\wandb\run-20250228_233740-5zcvrrkr\files\model-best\assets
wandb: Adding directory to artifact (c:\Users\vudut\OneDrive\Desktop\Python\MINI Project\wandb\run-20250228_233740-5zcvrrkr\files\model-best)... Done. 0.1s


1250/1250 [==============================] - 18s 15ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.4925
Epoch 4/10
1250/1250 [==============================] - 16s 13ms/step - loss: 0.6932 - accuracy: 0.5007 - val_loss: 0.6932 - val_accuracy: 0.4925
Epoch 5/10
1250/1250 [==============================] - 17s 13ms/step - loss: 0.6932 - accuracy: 0.5005 - val_loss: 0.6932 - val_accuracy: 0.4925
Epoch 6/10
1250/1250 [==============================] - 19s 15ms/step - loss: 0.6932 - accuracy: 0.4992 - val_loss: 0.6932 - val_accuracy: 0.4925
Epoch 7/10
1250/1250 [==============================] - 18s 14ms/step - loss: 0.6932 - accuracy: 0.5003 - val_loss: 0.6932 - val_accuracy: 0.4925
Epoch 8/10
1250/1250 [==============================] - 16s 13ms/step - loss: 0.6932 - accuracy: 0.4995 - val_loss: 0.6932 - val_accuracy: 0.4925
Epoch 9/10
1250/1250 [==============================] - 16s 13ms/step - loss: 0.6932 - accuracy: 0.4997 - val_loss: 0.6932 - val_accura

Test Accuracy,▁
Test Loss,▁
accuracy,█▇▄▇▆▁▅▂▃█
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁
val_loss,▃▁▁▄▆▄▅▅▄█
Test Accuracy,0.4925
Test Loss,0.69324
accuracy,0.50102
best_epoch,2


## Hyper Parameter Tuning

In [5]:
import tensorflow as tf
import wandb
import optuna
from tensorflow.keras import datasets, layers, models
from wandb.keras import WandbCallback

# Initialize W&B project
wandb.init(project="particle_cnn_tuning", name="optuna_sweep")

# Load dataset
dataset_path = "particle_images/processed"

def build_and_train_model(trial):
    # Define hyperparameters
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    dropout_rate = trial.suggest_uniform("dropout_rate", 0.2, 0.5)
    num_filters = trial.suggest_categorical("num_filters", [32, 64, 128])

    with tf.device('/GPU:0'):  # Use GPU for faster training
        train_ds = tf.keras.preprocessing.image_dataset_from_directory(
            dataset_path,
            validation_split=0.2,
            subset="training",
            seed=42,
            image_size=(64, 64),
            batch_size=batch_size
        )

        val_ds = tf.keras.preprocessing.image_dataset_from_directory(
            dataset_path,
            validation_split=0.2,
            subset="validation",
            seed=42,
            image_size=(64, 64),
            batch_size=batch_size
        )

    # Build CNN model
    model = models.Sequential([
        layers.Conv2D(num_filters, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(num_filters * 2, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(num_filters * 2, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(1, activation='sigmoid')
    ])

    # Compile model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train model with W&B logging
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=10,
        callbacks=[WandbCallback()]
    )

    # Evaluate model
    _, val_acc = model.evaluate(val_ds)
    wandb.log({"Validation Accuracy": val_acc})
    
    return val_acc  # Optuna maximizes this metric

# Optimize using Optuna
study = optuna.create_study(direction="maximize")  # Maximize validation accuracy
study.optimize(build_and_train_model, n_trials=10)

# Best hyperparameters
print("Best Hyperparameters:", study.best_params)

# Save best model hyperparameters to W&B
wandb.config.update(study.best_params)

wandb.finish()


[I 2025-02-28 23:41:34,821] A new study created in memory with name: no-name-063989ed-1843-4cac-8581-1fbb6efaaa7e
C:\Users\vudut\AppData\Local\Temp\ipykernel_29908\4024133043.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
C:\Users\vudut\AppData\Local\Temp\ipykernel_29908\4024133043.py:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform("dropout_rate", 0.2, 0.5)


Found 50000 files belonging to 2 classes.
Using 40000 files for training.
Found 50000 files belonging to 2 classes.
Using 10000 files for validation.
Epoch 1/10
  5/625 [..............................] - ETA: 26s - loss: 10.1237 - accuracy: 0.4375WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0185s vs `on_train_batch_end` time: 0.0222s). Check your callbacks.


624/625 [============================>.] - ETA: 0s - loss: 0.8720 - accuracy: 0.4985

INFO:tensorflow:Assets written to: c:\Users\vudut\OneDrive\Desktop\Python\MINI Project\wandb\run-20250228_234134-tnxxwrhl\files\model-best\assets


INFO:tensorflow:Assets written to: c:\Users\vudut\OneDrive\Desktop\Python\MINI Project\wandb\run-20250228_234134-tnxxwrhl\files\model-best\assets
wandb: Adding directory to artifact (c:\Users\vudut\OneDrive\Desktop\Python\MINI Project\wandb\run-20250228_234134-tnxxwrhl\files\model-best)... Done. 0.1s


...

[I 2025-03-01 00:23:27,788] Trial 8 finished with value: 0.492499977350235 and parameters: {'learning_rate': 0.0016747418044654263, 'batch_size': 16, 'dropout_rate': 0.45323760186571704, 'num_filters': 64}. Best is trial 0 with value: 0.492499977350235.


Found 50000 files belonging to 2 classes.
Using 40000 files for training.
Found 50000 files belonging to 2 classes.
Using 10000 files for validation.
Epoch 1/10
2500/2500 [==============================] - 24s 9ms/step - loss: 1.1828 - accuracy: 0.4970 - val_loss: 0.6944 - val_accuracy: 0.4925
Epoch 2/10
2500/2500 [==============================] - 24s 9ms/step - loss: 0.6934 - accuracy: 0.5001 - val_loss: 0.6943 - val_accuracy: 0.4925
Epoch 3/10
2500/2500 [==============================] - 23s 9ms/step - loss: 0.6934 - accuracy: 0.4971 - val_loss: 0.6943 - val_accuracy: 0.4925
Epoch 4/10
2500/2500 [==============================] - 22s 9ms/step - loss: 0.6934 - accuracy: 0.4992 - val_loss: 0.6941 - val_accuracy: 0.4925
Epoch 5/10
2500/2500 [==============================] - 22s 9ms/step - loss: 0.6934 - accuracy: 0.4983 - val_loss: 0.6940 - val_accuracy: 0.4925
Epoch 6/10
2500/2500 [==============================] - 23s 9ms/step - loss: 0.6934 - accuracy: 0.4984 - val_loss: 0.6940 - v

[I 2025-03-01 00:27:42,458] Trial 9 finished with value: 0.492499977350235 and parameters: {'learning_rate': 0.007535988281920984, 'batch_size': 16, 'dropout_rate': 0.26424947679762373, 'num_filters': 32}. Best is trial 0 with value: 0.492499977350235.


Best Hyperparameters: {'learning_rate': 4.3767651299882864e-05, 'batch_size': 64, 'dropout_rate': 0.403598174016896, 'num_filters': 128}


Validation Accuracy,▁▁▁▁▁▁▁▁▁▁
accuracy,▆▆▆▆▃██▂▄▄▆▆▆▆▄▆▄▃▅▄▃▄▆▆▆▃▁▂▃▃▄▆▄▆▄▃▄▄▃▃
epoch,▂▃▆▁▂▆▇█▂▄█▁▃▄█▄▆▇█▁▃▄▅▆▆█▁▃▅▆▄▅▆▇▁▃▇▁▅▆
loss,▁▁▁▁█▁▁▁▁▁▃▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▅▁▁
val_accuracy,▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁█▁▁▁▁▁▁▁▁▁▁
val_loss,▁▁▁▁▁▁▄▄▃▃▁▁▁▁▁▁▁▂▁▂▁▁▂▁▂▂▂▂▂▁▁▁▂▂▂▂▂▂██
Validation Accuracy,0.4925
accuracy,0.50042
best_epoch,5
best_val_loss,0.69313
epoch,9


In [6]:
print("Best Hyperparameters:", study.best_trial.params)

Best Hyperparameters: {'learning_rate': 4.3767651299882864e-05, 'batch_size': 64, 'dropout_rate': 0.403598174016896, 'num_filters': 128}


In [11]:
best_params = {
    "learning_rate": 4.3767651299882864e-05,
    "batch_size": 64,
    "dropout_rate": 0.403598174016896,
    "num_filters": 128
}


In [17]:
import tensorflow as tf
import wandb
from tensorflow.keras import layers, models
from tensorflow.keras.utils import image_dataset_from_directory

# ✅ Enable TensorFlow DirectML (Check if GPU is detected)
physical_devices = tf.config.list_physical_devices("GPU")
if physical_devices:
    print(f"✅ Using GPU: {physical_devices[0]}")  # No set_memory_growth to avoid error

# ✅ Initialize Weights & Biases (W&B) for tracking
wandb.init(project="particle_cnn_tuning", name="final_training", sync_tensorboard=True)

# ✅ Load Image Dataset
dataset_path = "particle_images"  # Ensure this folder contains the images
batch_size = 64  # From best hyperparameters
img_size = (64, 64)

train_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

# ✅ Apply performance optimizations
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# ✅ Best Hyperparameters (from Optuna tuning)
best_params = {
    "learning_rate": 4.3767651299882864e-05,
    "batch_size": 64,
    "dropout_rate": 0.403598174016896,
    "num_filters": 128
}

# ✅ Define CNN Model with Optimized Hyperparameters
model = models.Sequential([
    layers.Conv2D(best_params["num_filters"], (3, 3), activation="relu", input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(best_params["num_filters"] * 2, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(best_params["num_filters"] * 4, (3, 3), activation="relu"),
    layers.Flatten(),
    layers.Dropout(best_params["dropout_rate"]),
    layers.Dense(128, activation="relu"),
    layers.Dense(2, activation="softmax")  # Binary classification
])

# ✅ Compile Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=best_params["learning_rate"]),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# ✅ Train Model
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    callbacks=[wandb.keras.WandbCallback()]  # ✅ Track training with W&B
)

# ✅ Evaluate Model
test_loss, test_acc = model.evaluate(val_ds, verbose=2)
print(f"\n🎯 Final Test Accuracy: {test_acc:.4f}")

# ✅ Save Trained Model
model.save("particle_cnn_model.h5")
wandb.finish()  # ✅ Close W&B session


✅ Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


Found 100000 files belonging to 1 classes.
Using 80000 files for training.
Found 100000 files belonging to 1 classes.
Using 20000 files for validation.
Epoch 1/10
1250/1250 [==============================] - 65s 51ms/step - loss: 0.0020 - accuracy: 0.9998 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
1250/1250 [==============================] - 58s 46ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
1250/1250 [==============================] - 60s 48ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
1250/1250 [==============================] - 61s 49ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
1250/1250 [==============================] - 61s 49ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
1250/1250 [==============================] - 61s 49ms/step - loss: 0.00

accuracy,▁█████████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁
val_loss,▁▁▁▁▁▁▁▁▁▁
accuracy,1
epoch,9
loss,0
val_accuracy,1
val_loss,0
